In [16]:
import data_load as dl
import models
import matplotlib.pyplot as plt
import numpy as np

# from keras.models import Model
# from keras.layers import Input, Conv2D, ZeroPadding1D, Conv2DTranspose, UpSampling2D, Dense, BatchNormalization
# from keras.layers.pooling import MaxPooling2D, MaxPooling1D
from keras import optimizers
# import sklearn
# import tensorflow as tf


%matplotlib inline

In [2]:
path = r"C:\Users\Patrick\Datasets\sr\SR"
trial_paths = dl.combine_data(path)
trials = dl.generate_trial_objects(trial_paths)
processed_trials = dl.process_trial_data(trials)

C:\Users\Patrick\Anaconda3\lib\site-packages\openpyxl\reader\worksheet.py:318: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
#Smallest motion sample is 17559
# All G samples are 3030 long
# Take 2880 points from middle of 3030
# 3030 - 2880 = 150
#150/2 = 75
#Start each (Gs and downsampled motion on point 75) then sample out 2880

In [4]:
def synch_streams(G2, G3, G4, G5, motion):
    G2 = G2[75:-75]
    G3 = G3[75:-75]
    G4 = G4[75:-75]
    G5 = G5[75:-75]
    
    new_motion = motion[0:-1:6]
    new_motion = motion[75:(75+2880)]
    
    return G2, G3, G4, G5, new_motion

In [5]:
def generate_segments(G2, G3, G4, G5, motion):
    full_run_length = 2880
    num_segments = 400
    indices = np.random.randint(0,(full_run_length - 112), size=(num_segments))
    
    images_out = []
    motion_out = []
    
    for i in indices:
        G2_seg = G2[i:(i+112)]
        G3_seg = G3[i:(i+112)]
        G4_seg = G4[i:(i+112)]
        G5_seg = G5[i:(i+112)]
        motion_seg = motion[i:(i+112):2]
        
        new_image = np.array([[G2_seg],[G3_seg],[G4_seg],[G5_seg]])
        
        images_out.append(new_image)
        
        motion_out.append(np.asarray(motion_seg))
        
    return images_out, motion_out

In [6]:
X = []
y = []

for i in range(25):
    trial = processed_trials[i]
    G2_temp, G3_temp, G4_temp, G5_temp, motion_temp = synch_streams(trial.G2_AVG, trial.G3_AVG, trial.G4_AVG, trial.G5_AVG, trial.Motion_Conv)
    images, motion = generate_segments(G2_temp, G3_temp, G4_temp, G5_temp, motion_temp)
    
    for image in images:
        new_image=np.array(image)
        new_image.reshape(1,112,4)
        X.append(new_image)
    
    for entry in motion:
        y.append(np.array(entry))
        
X = np.array(X).reshape(10000,1,112,4)
y = np.array(y)

In [7]:
# trial = processed_trials[5]
# G3_temp, G4_temp, motion_temp = synch_streams(trial.G3_AVG, trial.G4_AVG, trial.Motion_Conv)
# images, motion = generate_segments(G3_temp, G4_temp, motion_temp)

# X = []
# y = []

# for image in images:
#     new_image=np.array(image)
#     new_image.reshape(1,112,2)
#     X.append(new_image)
    
# for entry in motion:
#     y.append(np.array(entry))
    


In [8]:
X.shape

(10000, 1, 112, 4)

In [9]:
y.shape

(10000, 56)

In [10]:
#X = np.expand_dims(X, axis=2)
#X = np.expand_dims(X, axis=3)

y = np.expand_dims(y, axis=1)
y.reshape(10000,1,56)
y = np.expand_dims(y, axis=3)

In [11]:
X.shape

(10000, 1, 112, 4)

In [12]:
y.shape

(10000, 1, 56, 1)

In [13]:
X = X/(np.max(abs(X)))

In [14]:
# inputs = Input(shape=(1,112,4), name="main_input")

# x1 = Conv2D(8, kernel_size=(1,3), padding="same")(inputs)
# x1 = MaxPooling2D(pool_size=(1,2), strides=(2))(x1)
# x1 = BatchNormalization(axis=1, momentum=0.99, epsilon=0.001, center=True, scale=True)(x1)

# x2 = Conv2D(16, kernel_size=(1,3),padding="same")(x1)
# x2 = MaxPooling2D(pool_size=(1,2), strides=(2))(x2)
# x2 = BatchNormalization(axis=1, momentum=0.99, epsilon=0.001, center=True, scale=True)(x2)

# x3 = Conv2D(32, kernel_size=(1,3),padding="same")(x2)
# x3 = MaxPooling2D(pool_size=(1,2), strides=(2))(x3)
# x3 = BatchNormalization(axis=1, momentum=0.99, epsilon=0.001, center=True, scale=True)(x3)

# x4 = Conv2D(64, kernel_size=(1,3),padding="same")(x3)
# x4 = MaxPooling2D(pool_size=(1,2), strides=(2))(x4)
# x4 = BatchNormalization(axis=1, momentum=0.99, epsilon=0.001, center=True, scale=True)(x4)

# x5 = Conv2D(128, kernel_size=(1,7))(x4)

# x6 = Dense(128)(x5)

# #Presently doing unpooling in hacked way (No filtering for max value, naive upsampling)
# # https://arxiv.org/pdf/1311.2901v3.pdf
# # https://stackoverflow.com/questions/44991470/using-tensorflow-layers-in-keras

# #Need to create custom keras layer to take mask from MaxPooling2D (similar to TF's max_pool_with_argmax)
# #mask is used in unpooling layer

# x7 = Conv2DTranspose(128, kernel_size=(1,7), padding='valid')(x6)
# x7 = UpSampling2D((1,2))(x7)
# x7 = BatchNormalization(axis=1, momentum=0.99, epsilon=0.001, center=True, scale=True)(x7)

# x8 = Conv2DTranspose(64, kernel_size=(1,3), padding='same')(x7)
# x8 = UpSampling2D((1,2))(x8)
# x8 = BatchNormalization(axis=1, momentum=0.99, epsilon=0.001, center=True, scale=True)(x8)

# x9 = Conv2DTranspose(32, kernel_size=(1,3), padding='same')(x8)
# x9 = UpSampling2D((1,2))(x9)
# x9 = BatchNormalization(axis=1, momentum=0.99, epsilon=0.001, center=True, scale=True)(x9)

# x10 = Conv2DTranspose(16, kernel_size=(1,3), padding='same')(x9)
# x10 = BatchNormalization(axis=1, momentum=0.99, epsilon=0.001, center=True, scale=True)(x10)

# x11 = Conv2DTranspose(4, kernel_size=(1,3), padding='same')(x10)
# x11 = BatchNormalization(axis=1, momentum=0.99, epsilon=0.001, center=True, scale=True)(x11)

# x11 = Conv2DTranspose(2, kernel_size=(1,3), padding='same')(x11)
# x11 = Conv2DTranspose(1, kernel_size=(1,3), padding='same')(x11)
# x12 = BatchNormalization(axis=1, momentum=0.99, epsilon=0.001, center=True, scale=True)(x11)

# model2 = Model(inputs=inputs, outputs=x12)

# sgd = optimizers.SGD(lr=0.01, momentum=0.9, decay=0.0005)

# model2.compile(optimizer=sgd, loss = 'mean_squared_error', metrics = ['mse'])

In [17]:
model2 = models.FCN(4,1)

In [ ]:
for layer in model2.layers:
    print(layer.output_shape)

In [ ]:
X_small = X[0:100]
y_small = y[0:100]

In [ ]:
model2.fit(X_small, y_small, epochs=1000)

In [ ]:
testno = 99
testX = np.expand_dims(X[testno],0)
testy = np.squeeze(y[testno])

In [ ]:
y_pred = model2.predict(testX)
p = np.squeeze(y_pred)

In [ ]:
plt.figure()
plt.plot(testy)
plt.plot(p)

In [19]:
trial_paths[1]

['C:\\Users\\Patrick\\Datasets\\sr\\SR\\1.Data\\20180220.3\\20180220.3.3',
 'C:\\Users\\Patrick\\Datasets\\sr\\SR\\1.Data2\\20180220.3\\20180220.3.3\\1.Traces']